In [ ]:
from glob import glob
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
ol=['ol0','ol1','ol2','ol3']
rl=['rl0','rl1','rl2']

In [ ]:
import pandas as pd

datadct = {}

for o in ol:
    datadct[o] = {}
    for r in rl:
        try:
            df = pd.read_json(f'{o}_{r}/A2_HWE-CNOT_results.json', orient='index').T
            df.index = pd.MultiIndex.from_tuples([(o, r)])  # Correct MultiIndex usage
            print(df)
            datadct[o][r] = df
        except FileNotFoundError:
            continue  # Skip missing files
        except Exception as e:
            print(f"Error processing {o}_{r}: {e}")  # Print error for debugging


In [ ]:
df = pd.concat([pd.concat([v for v in v1.values()]).reset_index() for v1 in datadct.values()])
meltdf = df.rename(columns={'R2_train':"Train",'R2_test':'Test',"level_0":"Optimization Level","level_1":"Resilience Level"}).melt(id_vars=['Optimization Level','Resilience Level'],value_vars=['Train','Test'])

In [ ]:
df

In [ ]:
sns.lineplot(df,y='R2_train',hue='level_0',x='level_1')

In [ ]:
meltdf['Optimization Level'] = [int(i.strip('ol')) for i in meltdf['Optimization Level']]
meltdf['Resilience Level'] = [int(i.strip('rl')) for i in meltdf['Resilience Level']]

In [ ]:
# Create FacetGrid with subplots for each Optimization Level
g = sns.FacetGrid(meltdf, col="Optimization Level", col_wrap=2, height=4, sharey=True)

# Map the line plot with markers that follow hue style
g.map_dataframe(
    sns.lineplot, 
    x="Resilience Level", 
    y="value", 
    hue="variable", 
    style="variable",  # Ensures markers match hue
    markers=True,  
    palette=sns.color_palette("Paired", 2)
)

# Adjust legends, labels, and axis limits
g.add_legend()
g.set_axis_labels("Resilience Level", r"R$^{2}$")  # LaTeX formatting
g.set(ylim=(0, 1), xticks=[0, 1, 2])  # Fix y-range and x-ticks

# Tight layout for better spacing
plt.tight_layout()

# Save the figure as PNG with high quality
plt.savefig(os.path.join(os.path.expanduser('~'),'qregress/images/DDCC/opt_res_fakequebec.png'), dpi=300, bbox_inches="tight")

# Show the plot
plt.show()